<a href="https://colab.research.google.com/github/tarashagarwal/Analyzing-a-Global-Supply-Chain-Network/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [228]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sn
import time

In [229]:
#Conver MNIST image files to Tensors (#of Imhges, Height, Width and Color Channels)
transform = transforms.ToTensor()


In [230]:
#Train Data
train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transform)

In [231]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [232]:
#Test Data
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transform)

In [233]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [234]:
# Create a small batch size for Images ... lets say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=10, shuffle=False)


In [235]:
#Define out CNN MOdel
#Describe convolution layer and what its doing (2 convolutional layer)
#This is just an
conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)

In [236]:
#Grab 1 MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
    break

In [237]:
X_train.shape
torch.Size([1, 28, 28])

torch.Size([1, 28, 28])

In [238]:
x = X_train.view(1, 1, 28, 28)

In [239]:
x = F.relu(conv1(x))

In [240]:
x.shape#There is no paddign defined so some data is being dropped

torch.Size([1, 6, 26, 26])

In [241]:
#pass through the pooling layer
x = F.max_pool2d(x,2,2)

In [242]:
x.shape #26/2 = 13

torch.Size([1, 6, 13, 13])

In [243]:
# Do out second convolution layer
x = F.relu(conv2(x))

In [244]:
x.shape #we didnt set padding so we lose 2 pixels outside of the image

torch.Size([1, 16, 11, 11])

In [245]:
x = F.max_pool2d(x,2,2)

In [246]:
x.shape

torch.Size([1, 16, 5, 5])

In [247]:
from os import fchdir
#Model Class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1,6,3,1)
    self.conv2 = nn.Conv2d(6,16,3,1)
    #Fully connected Layer is also needed
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x,2,2) #2x2 kernal and stride 2
    # Second Pass
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x,2,2) # 2x2 kernel and stride 2

    #Re-View to flatten it out
    x = x.view(-1, 16*5*5) #negeative so that you can vary the batch size

    #Fully Connected Layers
    X = F.relu(self.fc1(x))
    X = F.relu(self.fc2(x))
    X = self.fc3(x)

    return F.log_softmax(x, dim=1)

In [248]:
#create a instance of the mode
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [249]:
#Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #smaller the learning rate the longer its gona take to train
